
Created on Sat Sep 24 17:40:00 2022

@author: JESUS ALEJANDRO COLIN VILCHIS

# target:
  ## Analysis of the gisaid data, until August 31, 2022


In [2]:
# Pandas
import pandas as pd

# Python
import warnings
import os

from concurrent.futures import ThreadPoolExecutor

# Plotly
import plotly.graph_objects as go
import plotly.express as px
import datetime

# Developed
from utils import Files as f
from utils import Counter as c
from utils import counter, autoadjust, add_p_value_annotation
from utils import get_voc, fromisocalendar

Cleaning of library alerts

In [2]:
warnings.filterwarnings('ignore')

Loading, preprocessing, normalization and data cleaning.

In [3]:
files = os.listdir("files")

# Reading
original_file = pd.read_csv("files/gisaid_hcov-19_2022_08_17_04.tsv",sep="\t")
dictionaries = f.load_dict(dir = "json")

# Preprocessing and standardization
completed_file = f.complete(original_file)
normalized_file = f.normalize(completed_file , dictionaries)

# Tables are filtered according to age
df_under18 = f.filter_age(normalized_file,"under18")
df_over18 = f.filter_age(normalized_file,"over18")


set()


In [4]:
start = fromisocalendar(*min(normalized_file.date))
end = fromisocalendar(*max(normalized_file.date))
print(f"Records span between the {start} to {end}")

Records span between the 27/02/2020 to 21/09/2022


The size of the recordsets according to their age are as follows.

In [5]:
n_under18 = len(df_under18)
n_over18 = len(df_over18)

print(
    "Under_18: \t", n_under18 ,"\n", 
    "Over_18: \t", n_over18 ,"\n", 
    "Total: \t", len(normalized_file)
    )

Under_18: 	 10306 
 Over_18: 	 66538 
 Total: 	 76844


Counting mutations, and calculating percentages

In [6]:
if "table_mutation_count.csv" not in files:
    # Mutations grouped by protein of occurrence are counted
    df_count_all = counter(
        normalized_file,
        dictionaries["proteins"])
    
    df_count_under18 = counter(
        df_under18,
        dictionaries["proteins"])
    
    df_count_over18 = counter(
        df_over18,
        dictionaries["proteins"])


    # column name change
    mk_table = lambda df:pd.concat(
        [df[k].rename(columns = {k:'mutation'}) 
         for k in df.keys()] 
        ).set_index('mutation')
    
    df_mutations_total = mk_table(df_count_all)
    df_mutations_under18 = mk_table(df_count_under18)
    df_mutations_over18 = mk_table(df_count_over18)

    # column name change
    rname = lambda df,txt="" : df.rename(columns = {i:i+txt if i!= 'mutation'else i for i in df.columns})
    df_m_total= rname(df_mutations_total)
    df_m_under18= rname(df_mutations_under18,"_under18")
    df_m_over18= rname(df_mutations_over18,"_over18")

    # Join the tables
    df_mutations = df_m_total.join(df_m_under18).join(df_m_over18)
    
    # Percentage calculation
    df_mutations['percentage_under18'] = [(df_mutations['count_under18'].iloc[i]/n_under18)*100 for i in range(len(df_mutations))]
    df_mutations['percentage_over18'] = [(df_mutations['count_over18'].iloc[i]/n_over18)*100  for i in range(len(df_mutations))]
    df_mutations = df_mutations[['change','position','count','count_under18','percentage_under18','count_over18','percentage_over18','full']].fillna(0)

    # save the results
    df_mutations.to_csv('files/table_mutation_count.csv')
    
else:
    
    #save the results
    df_mutations = pd.read_csv("files/table_mutation_count.csv")


In [7]:
print(f'A total of {sum(df_mutations["count"])} amino acid (aa) changes were documented {len(df_mutations["count"])} variants')

A total of 2742529 amino acid (aa) changes were documented 18089 variants


Most frequent mutations and their percentage that they represent with respect to the total of mutations

In [8]:
df_mutations.sort_values(by='count', ascending=False).head(10)

,mutation,change,position,count,count_under18,percentage_under18,count_over18,percentage_over18,full
6113,NSP12_P323L,323,0,75993,10234.0,99.301378,65759.0,98.829240,NSP12
431,Spike_D614G,614,S1D domain,75989,10167.0,98.651271,65822.0,98.923923,Spike
16348,NSP4_T492I,492,0,59123,7112.0,69.008345,52011.0,78.167363,NSP4
2775,Spike_T478K,478,S1B domain,55771,7002.0,67.941005,48769.0,73.294959,Spike
4391,N_R203K,203,Enlazador central desordenado predicho (LINK),42369,4378.0,42.480109,37991.0,57.096697,N
3858,N_G204R,204,Enlazador central desordenado predicho (LINK),42275,4377.0,42.470406,37898.0,56.956927,N
2049,Spike_P681H,681,Protease cleavage site,41267,4005.0,38.860858,37262.0,56.001082,Spike
848,Spike_G142D,142,S1A domain,35740,5478.0,53.153503,30262.0,45.480778,Spike
17087,NSP6_G107del,107,0,35018,3628.0,35.202794,31390.0,47.176050,NSP6
17361,NSP6_S106del,106,0,34998,3625.0,35.173685,31373.0,47.150500,NSP6


In [9]:
count = lambda data,key:data.groupby([key]).size().reset_index(name='patients').sort_values(by=['patients'],ascending=False)


variants of mutations by proteins (Does not represent the sum of each, only the variants)

In [10]:
count(df_mutations,'full')

,full,patients
11,NSP3,4070
18,Spike,3435
10,NSP2,1685
5,NSP12,1322
2,N,1103
12,NSP4,900
7,NSP14,865
6,NSP13,776
8,NSP15,629
14,NSP6,578


In [11]:
positions = [614, 681 , 478]
count(df_mutations.query(f"change in {positions}"),'full')

,full,patients
3,Spike,16
1,NSP2,7
2,NSP3,7
0,NSP12,4


In [12]:
df_mutations.query(f"change in {positions}")

,mutation,change,position,count,count_under18,percentage_under18,count_over18,percentage_over18,full
431,Spike_D614G,614,S1D domain,75989,10167.0,98.651271,65822.0,98.923923,Spike
2048,Spike_P681C,681,Protease cleavage site,1,0.0,0.000000,1.0,0.001503,Spike
2049,Spike_P681H,681,Protease cleavage site,41267,4005.0,38.860858,37262.0,56.001082,Spike
2050,Spike_P681L,681,Protease cleavage site,11,1.0,0.009703,10.0,0.015029,Spike
2051,Spike_P681R,681,Protease cleavage site,26350,4116.0,39.937900,22234.0,33.415492,Spike
2052,Spike_P681S,681,Protease cleavage site,3,0.0,0.000000,3.0,0.004509,Spike
2053,Spike_P681Y,681,Protease cleavage site,5,0.0,0.000000,5.0,0.007515,Spike
2054,Spike_P681del,681,Protease cleavage site,6,2.0,0.019406,4.0,0.006012,Spike
2773,Spike_T478E,478,S1B domain,5,1.0,0.009703,4.0,0.006012,Spike
2774,Spike_T478I,478,S1B domain,5,0.0,0.000000,5.0,0.007515,Spike


In [13]:
if "tabla_mutations_variant.csv" not in files:
    
    
    with ThreadPoolExecutor(max_workers = 3) as executor:
        f1 = executor.submit(c.properties_mutations,normalized_file,df_mutations['change'].tolist())
        f2 = executor.submit(c.properties_mutations,df_under18,df_mutations['change'].tolist())
        f3 = executor.submit(c.properties_mutations,df_over18,df_mutations['change'].tolist())

    table_mutations_variant = f1.result()
    table_mutations_variant_under = f2.result() 
    table_mutations_variant_over18 = f3.result() 

    # Dictionaries are converted into dataframes
    table_mutations_variant = pd.DataFrame(table_mutations_variant)
    table_mutations_variant_under18 = pd.DataFrame(table_mutations_variant_under)
    table_mutations_variant_over18 = pd.DataFrame(table_mutations_variant_over18)

    # Join the tables
    table_mv = pd.concat(
        [
            table_mutations_variant.transpose(), 
            table_mutations_variant_under18.transpose(), 
            table_mutations_variant_over18.transpose()], 
        axis=1
        )
    
    #save the results
    table_mv.to_csv("files/tabla_mutations_variant.csv")
    
else:
    
    #save the results
    table_mv = pd.read_csv("files/tabla_mutations_variant.csv")

# Graphics

From the age distributions for each variant, a box plot is generated.

In [14]:
fig = go.Figure()

# For each variant
for vt in normalized_file["variant_type"].unique():
    
    fig.add_trace(
        go.Box(
            y=normalized_file[
                normalized_file["variant_type"] == vt]['age'],
            name=vt,
            boxpoints='outliers'
        ))
    
fig = add_p_value_annotation(fig, [[0,1], [0,2], [0,3], [0,4], [0,5]])

fig = fig.update_layout(
    autosize=False,
    width=1200,
    showlegend=True,
    template="plotly_white"
    )

fig.show()

Cleaning week no data uploaded
* This is for display only

In [15]:


voc = get_voc(normalized_file)
voc = pd.DataFrame(voc)
voc.iloc[106] = voc.iloc[105]
voc = voc.iloc[8:]

100 Percent Stacked Area Chart from variant counts

In [16]:
df = voc.divide(voc.sum(axis=1), axis=0)
df.to_csv("files/variant_semana.csv")
fig = px.area(
    df,
    line_shape="spline",
    template="plotly_white"
    )
fig.show()


Cleaning week no data uploaded
* This is for display only

In [17]:
from utils import get_clado
clado = get_clado(normalized_file)
clado = pd.DataFrame(clado)
clado.iloc[106] = clado.iloc[105]#[104:107]
clado = clado.iloc[8:]
clado.to_csv("files/clado_semana.csv")

100 Percent Stacked Area Chart from clade counts

In [18]:
fig = px.area(
    clado.divide(
        clado.sum(axis=1), 
        axis=0),
    line_shape="spline",
    template="plotly_white"
    )
fig.show()

In [19]:
from utils import count_representative_state, count_representative_country

writer =  pd.ExcelWriter('files/estados.xlsx')
count_representative_state(
  writer,
  normalized_file,
  dictionaries
  )


count_representative_country(
    writer,
    normalized_file,
)

writer.save()

In [20]:

week_state = dict()

for i in dictionaries["unique_states_types"].values():
  week_state[i] = pd.read_excel(
    open('files/estados.xlsx', 'rb'), 
    header = 1 ,
    sheet_name=str(i))[['week','predominant']]

In [21]:
from utils import get_states

states = get_states(normalized_file, dictionaries)
df_states = pd.DataFrame(states,columns=['week_cont', 'state_key' ,'variant_type'])

In [22]:
df_states.groupby('variant_type').size()

variant_type
Alpha              21
Beta                1
Delta             758
Gamma              39
None             1651
Omicron          1066
Other linages     985
dtype: int64

In [23]:
fig = px.scatter(
    df_states.query('state_key != "Extra"'), 
    y='state_key', 
    x= 'week_cont', 
    color='variant_type', 
    size_max=60,
    template='plotly_white', 
    title="variante predominante por Estado",
    color_discrete_sequence=px.colors.qualitative.Vivid
    )

fig.update_layout(yaxis={
                    'categoryorder': 'array', 
                    'categoryarray': [
                        'Baja California','Baja California Sur',
                        'Sonora','Chihuahua','Coahuila',
                        'Nuevo Leon','Colima','Chiapas',
                        'Tamaulipas','Aguascalientes','Hidalgo',
                        'Guanajuato','Durango','Jalisco',
                        'Sinaloa','Ciudad de Mexico',
                        'Estado de Mexico','Michoacan',
                        'Morelos','Nayarit','Oaxaca',
                        'Puebla','Queretaro','San Luis Potosi',
                        'Guerero','Tabasco','Tlaxcala',
                        'Veracruz','Zacatecas','Campeche',
                        'Quintana Roo','Yucatan'
                        ]},
    width=1000,
    height=1000)

fig.show()

In [24]:
df_states.to_csv("files/estados_por_puntos.csv")

In [25]:
df_states.groupby(['state_key','variant_type']).size().to_csv("files/estados_cuenta.csv")

In [26]:
path = "files/220901COVID19MEXICO.csv"
df_sinav = pd.read_csv(path)
df_sinav = df_sinav[df_sinav['FECHA_INGRESO'].notna()]
df_sinav = df_sinav.query('CLASIFICACION_FINAL == 3 and EDAD < 18 ')
df_sinav.head()

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,OTRO_CASO,TOMA_MUESTRA_LAB,RESULTADO_LAB,TOMA_MUESTRA_ANTIGENO,RESULTADO_ANTIGENO,CLASIFICACION_FINAL,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI
137,2022-09-01,349086,1,12,32,2,32,32,17,1,...,2,1,1,1,1,3,99,México,97,97
394,2022-09-01,9306cb,2,12,9,1,9,9,7,1,...,2,2,97,1,1,3,99,México,97,97
441,2022-09-01,575c72,1,12,9,2,9,9,13,1,...,2,2,97,1,1,3,99,México,97,97
504,2022-09-01,de0dac,1,12,21,2,21,21,156,1,...,2,1,1,1,2,3,99,México,97,97
651,2022-09-01,61fddd,2,12,9,2,9,9,8,1,...,2,2,97,1,1,3,99,México,97,97


In [27]:
df_sinav['Year'] = [datetime.datetime.strptime(i,'%Y-%m-%d').year for i in df_sinav['FECHA_INGRESO']]
df_sinav['date'] = [datetime.datetime.strptime(i,'%Y-%m-%d').isocalendar() for i in df_sinav['FECHA_INGRESO']]

delay = {
    2020:0,
    2021:53,
    2022:106
    }

df_sinav["week"] = [df_sinav.date.iloc[i][1]  for i in range(len(df_sinav))]
df_sinav["week_cont"] = [df_sinav.date.iloc[i][1] + delay[df_sinav.date.iloc[i][0]] for i in range(len(df_sinav))]

In [28]:
len(df_sinav)

208431

In [29]:
df_sinav['TIPO_PACIENTE_AMP'] = [
    df_sinav['TIPO_PACIENTE'].iloc[i] if df_sinav['FECHA_DEF'].iloc[i] == '9999-99-99' else 3  
    for i in range(len(df_sinav))
    ]
df_sinav['UCI_INTUBADO'] = [1 if df_sinav['UCI'].iloc[i] == 1 or df_sinav['INTUBADO'].iloc[i] == 1 else 2 
                            for i in range(len(df_sinav))]
df_sinav['EMBARAZO_bool'] = [1 if df_sinav['EMBARAZO'].iloc[i] == 1  else 2 for i in range(len(df_sinav))]

In [30]:
def calculate_sinav(df_sinav):
    table = list()
    for i in range(len(df_sinav)):
        try:
            val = normalized_file.query(
                f" state_key == { df_sinav['ENTIDAD_RES'].iloc[i] } and week_cont == { df_sinav['week_cont'].iloc[i] }"
                ).groupby(
                ['variant_type']
                    ).size().transform(lambda x: x/x.sum())
            table.append(val[val>0.5].keys()[0])
        except IndexError:
            table.append('None')
        except Exception as ex:
            table.append('None')
            print(ex)
    return table

In [31]:
df_sinav = df_sinav[1:]
n = len(df_sinav)/5

In [ ]:
with ThreadPoolExecutor(max_workers=10) as executor:
    futures = list()
    futures.append(executor.submit(calculate_sinav,df_sinav.iloc[:int(n)]))
    futures.append(executor.submit(calculate_sinav,df_sinav.iloc[int(n):int(2*n)]))
    futures.append(executor.submit(calculate_sinav,df_sinav.iloc[int(2*n):int(3*n)]))
    futures.append(executor.submit(calculate_sinav,df_sinav.iloc[int(3*n):int(4*n)]))
    futures.append(executor.submit(calculate_sinav,df_sinav.iloc[int(4*n):]))

In [ ]:
r = list()
for i in futures:
    r = r + i.result()
df_sinav['Predominant'] = r


In [ ]:
df_sinav.to_csv("files/sinav_predominant.csv")

In [4]:
df = pd.read_excel('files/sinav.xlsx')
table = df.groupby(['week_cont','predominant','gpoped']).size().reset_index(name="paciente")
table.set_index(['week_cont','predominant','gpoped']).unstack(level=0).fillna(0)

paciente                                         \
week_cont                     10   11   12   13    14    15    16    17    
predominant   gpoped                                                       
Alpha         Adolescent      0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
              Child           0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
              Infant          0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
              Prescholar      0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
Delta         Adolescent      0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
              Child           0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
              Infant          0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
              Prescholar      0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
Gamma         Adolescent      0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
              Child           0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
              Infant          0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
              Prescholar      0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
Omicron       Adolescent      0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
              Child           0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
              Infant          0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
              Prescholar      0.0  0.0  0.0  0.0   0.0   0.0   0.0   0.0   
Other linages Adolescent      1.0  1.0  5.0  6.0   5.0  21.0  34.0  50.0   
              Child           0.0  1.0  7.0  5.0  11.0   6.0  25.0  41.0   
              Infant          0.0  0.0  1.0  0.0   2.0  13.0  19.0  33.0   
              Prescholar      0.0  1.0  0.0  5.0   3.0   6.0   9.0  13.0   

                                       ...                               \
week_cont                  18     19   ...     139    140    141    142   
predominant   gpoped                   ...                                
Alpha         Adolescent   0.0    0.0  ...     0.0    0.0    0.0    0.0   
              Child        0.0    0.0  ...     0.0    0.0    0.0    0.0   
              Infant       0.0    0.0  ...     0.0    0.0    0.0    0.0   
              Prescholar   0.0    0.0  ...     0.0    0.0    0.0    0.0   
Delta         Adolescent   0.0    0.0  ...     0.0    0.0    0.0    0.0   
              Child        0.0    0.0  ...     0.0    0.0    0.0    0.0   
              Infant       0.0    0.0  ...     0.0    0.0    0.0    0.0   
              Prescholar   0.0    0.0  ...     0.0    0.0    0.0    0.0   
Gamma         Adolescent   0.0    0.0  ...     0.0    0.0    0.0    0.0   
              Child        0.0    0.0  ...     0.0    0.0    0.0    0.0   
              Infant       0.0    0.0  ...     0.0    0.0    0.0    0.0   
              Prescholar   0.0    0.0  ...     0.0    0.0    0.0    0.0   
Omicron       Adolescent   0.0    0.0  ...  1265.0  802.0  672.0  561.0   
              Child        0.0    0.0  ...  1027.0  614.0  442.0  395.0   
              Infant       0.0    0.0  ...   689.0  400.0  286.0  186.0   
              Prescholar   0.0    0.0  ...   479.0  284.0  186.0  182.0   
Other linages Adolescent  73.0  117.0  ...     0.0    0.0    0.0   28.0   
              Child       61.0   76.0  ...     0.0    0.0    0.0   22.0   
              Infant      44.0   63.0  ...     0.0    0.0    0.0   11.0   
              Prescholar  18.0   24.0  ...     0.0    0.0    0.0    5.0   

                                                                   
week_cont                   143    144    145    146    147   148  
predominant   gpoped                                               
Alpha         Adolescent    0.0    0.0    0.0    0.0    0.0   0.0  
              Child         0.0    0.0    0.0    0.0    0.0   0.0  
              Infant        0.0    0.0    0.0    0.0    0.0   0.0  
              Prescholar    0.0    0.0    0.0    0.0    0.0   0.0  
Delta         Adolescent    0.0    0.0    0.0    0.0    0.0   0.0  
      

In [5]:
table

,week_cont,predominant,gpoped,paciente
0,10,Other linages,Adolescent,1
1,11,Other linages,Adolescent,1
2,11,Other linages,Child,1
3,11,Other linages,Prescholar,1
4,12,Other linages,Adolescent,5
...,...,...,...,...
876,147,Omicron,Prescholar,38
877,148,Omicron,Adolescent,13
878,148,Omicron,Child,13
879,148,Omicron,Infant,3
